In [1]:
import sys
sys.path.append("python/")

import sympy as sp
import numdifftools as nd
import numpy as np
import scipy.optimize as opt

import datlib
import symbase
import Z2num
import Z2sym

# Ферромагнетик в симметричном случае

## Выражение для $Z^{(2)}$

### Интеграл

In [2]:
h, l = Z2sym.h, Z2sym.l
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t, u = sp.symbols("t u")

In [3]:
expr = sp.Integral(sp.exp(t**2/(2*l) + s1*t), (t, 0, h+s2*l))
expr = expr.transform(t, (t-s1*l, t)).expand()
expr = expr.transform(t, (t*sp.sqrt(2*l), t)).factor()
display(expr)

expr = expr.replace(sp.Integral, lambda *args: sp.exp(args[1][2]**2)).expand()
expr = expr*sp.exp(-(h**2+l**2)/(2*l))
expr.expand()

sqrt(2)*sqrt(\lambda)*exp(-\lambda/2)*Integral(exp(t**2), (t, sqrt(2)*\sigma_1*sqrt(\lambda)/2, sqrt(2)*(\sigma_1*\lambda + \sigma_2*\lambda + h)/(2*sqrt(\lambda))))

sqrt(2)*sqrt(\lambda)*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)

### Производные $\cal{F}^{(2)}$

In [4]:
expr = s1*s2*Z2sym._sigmas_expr_E_symmetrical*Z2sym._sigmas_expr_cF2_symmetrical
expr = expr.diff(l).expand()
symbase.collect_sigmas(expr, Z2sym._sigmas_terms_symmetrical, [s1, s2])

{exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)*F2(sqrt(2)*\sigma_1*sqrt(\lambda)/2 + sqrt(2)*\sigma_2*sqrt(\lambda)/2 + sqrt(2)*h/(2*sqrt(\lambda))): {(1,
   1): (-2*\lambda**2 + h**2)/(2*\lambda**2)},
 exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda): {(0,
   1): sqrt(2)/(4*sqrt(\lambda)),
  (1, 0): sqrt(2)/(4*sqrt(\lambda)),
  (1, 1): -sqrt(2)*h/(4*\lambda**(3/2))}}

In [5]:
display(Z2sym.E_symmetrical(h, l, 1, 1).diff(h).rewrite(sp.exp))
display(Z2sym.E_symmetrical(h, l, 1, 1).rewrite(sp.exp).diff(h))

2*exp(\lambda)*exp(2*h) - 2*exp(\lambda)*exp(-2*h)

2*exp(\lambda)*exp(2*h) - 2*exp(\lambda)*exp(-2*h)

In [6]:
display(Z2sym.cF2_symmetrical(h, l).diff(l).rewrite(sp.exp).expand())
display(Z2sym.cF2_symmetrical(h, l).rewrite(sp.exp).diff(l).expand())

-exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda))) + exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda))) + 2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda))) + h**2*exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda)))/\lambda**2 + sqrt(2)*exp(\lambda)*exp(2*h)/(2*sqrt(\lambda)) - sqrt(2)*exp(\lambda)*exp(-2*h)/(2*sqrt(\lambda)) - sqrt(2)*h*exp(\lambda)*exp(2*h)/(4*\lambda**(3/2)) - sqrt(2)*h*exp(\lambda)*exp(-2*h)/(4*\lambda**(3/2)) + sqrt(2)*h*exp(-\lambda)/(2*\lambda**(3/2))

-exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda))) + exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda))) + 2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda))) + h**2*exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda)))/\lambda**2 + sqrt(2)*exp(\lambda)*exp(2*h)/(2*sqrt(\lambda)) - sqrt(2)*exp(\lambda)*exp(-2*h)/(2*sqrt(\lambda)) - sqrt(2)*h*exp(\lambda)*exp(2*h)/(4*\lambda**(3/2)) - sqrt(2)*h*exp(\lambda)*exp(-2*h)/(4*\lambda**(3/2)) + sqrt(2)*h*exp(-\lambda)/(2*\lambda**(3/2))

### Численная проверка

In [7]:
h, l = 1, 1.4
print(Z2num.calc_Z2_integrate(0, h, h, l)*np.exp(-2*h-l))
print(Z2num.calc_from_coeffs_symmetrical(h, l).Z2_norm)

11.129141015312976
11.129142009015116


In [8]:
h, l = Z2sym.h, Z2sym.l
cF2_diff_h = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(h), Z2sym.num_module)
cF2_diff_l = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(l), Z2sym.num_module)

args = 1, 1.8
gradient = nd.Gradient(lambda args: Z2num.cF2_symmetrical(*args))(args)
print(gradient[0])
print(cF2_diff_h(*args))
print()
print(gradient[1])
print(cF2_diff_l(*args))

18.342584883167927
18.3425941684832

8.179230830337767
8.179237578451808


In [9]:
h, l = Z2sym.h, Z2sym.l
sigmas_args = 1, 1
E_diff_h = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(h), Z2sym.num_module)
E_diff_l = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(l), Z2sym.num_module)

args = 1, 1.8
sigmas_args = list(map(bool, sigmas_args))

gradient = nd.Gradient(lambda args: Z2num.E_symmetrical(*args, *sigmas_args))(args)

print(gradient[0])
print(E_diff_h(*args))
print()
print(gradient[1])
print(E_diff_l(*args))

87.76490748044182
87.76490748044566

45.850513022822085
45.85051302282197


## Моменты

### Символьные вычисления

In [10]:
h, l = Z2sym.h, Z2sym.l
Z2_symmetrical = sp.sqrt(2)*(2*sp.pi)**2*Z2sym.cF2_symmetrical(h, l)/(h*sp.sqrt(l))

In [11]:
m, eta = Z2sym.m, Z2sym.eta
Z2 = Z2_symmetrical
Z2

4*sqrt(2)*pi**2*cF2_symmetrical(h, \lambda)/(sqrt(\lambda)*h)

In [12]:
expr_m = sp.log(Z2).diff(h)/2
expr_m = expr_m.expand()
display(sp.Eq(m, expr_m))
sp.print_latex(sp.Eq(m, expr_m))

Eq(\left<m\right>, -1/(2*h) - h/(2*\lambda) + sqrt(2)*E_symmetrical(h, \lambda, 1, 1)/(4*sqrt(\lambda)*cF2_symmetrical(h, \lambda)))

\left<m\right> = - \frac{1}{2 h} - \frac{h}{2 \lambda} + \frac{\sqrt{2} {\cal E}_{12}\left(h, \lambda\right)}{4 \sqrt{\lambda} {\cal F}^{(2)}\left(h, \lambda\right)}


In [13]:
expr_eta = sp.log(Z2).diff(l)
expr_eta = expr_eta.expand()
display(sp.Eq(eta, expr_eta))
sp.print_latex(sp.Eq(eta, expr_eta))

Eq(\left<\eta\right>, -1 - 1/(2*\lambda) + h**2/(2*\lambda**2) + sqrt(2)*E_symmetrical(h, \lambda, 1, 0)/(2*sqrt(\lambda)*cF2_symmetrical(h, \lambda)) - sqrt(2)*h*E_symmetrical(h, \lambda, 1, 1)/(4*\lambda**(3/2)*cF2_symmetrical(h, \lambda)))

\left<\eta\right> = -1 - \frac{1}{2 \lambda} + \frac{h^{2}}{2 \lambda^{2}} + \frac{\sqrt{2} {\cal E}_{1}\left(h, \lambda\right)}{2 \sqrt{\lambda} {\cal F}^{(2)}\left(h, \lambda\right)} - \frac{\sqrt{2} h {\cal E}_{12}\left(h, \lambda\right)}{4 \lambda^{\frac{3}{2}} {\cal F}^{(2)}\left(h, \lambda\right)}


In [14]:
_E12, _E1 = Z2sym.E_symmetrical(h, l, 1, 1), Z2sym.E_symmetrical(h, l, 1, 0)
_E12_E1 = sp.solve([m-expr_m, eta-expr_eta], _E12, _E1)
_expr_E12 = _E12_E1[_E12].factor()
_expr_E1 = _E12_E1[_E1].factor()

_expr_cF2_1 = sp.solve(_E12-_expr_E12, Z2sym.cF2_symmetrical(h, l))[0]
_expr_cF2_2 = sp.solve(_E1-_expr_E1, Z2sym.cF2_symmetrical(h, l))[0]

display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))


Eq(cF2_symmetrical(h, \lambda), sqrt(2)*sqrt(\lambda)*h*E_symmetrical(h, \lambda, 1, 1)/(2*(2*\lambda*\left<m\right>*h + \lambda + h**2)))

{\cal F}^{(2)}\left(h, \lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} h {\cal E}_{12}\left(h, \lambda\right)}{2 \left(2 \lambda \left<m\right> h + \lambda + h^{2}\right)}


Eq(cF2_symmetrical(h, \lambda), sqrt(2)*sqrt(\lambda)*E_symmetrical(h, \lambda, 1, 0)/(2*(\lambda*\left<\eta\right> + \lambda + \left<m\right>*h + 1)))

{\cal F}^{(2)}\left(h, \lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} {\cal E}_{1}\left(h, \lambda\right)}{2 \left(\lambda \left<\eta\right> + \lambda + \left<m\right> h + 1\right)}


In [15]:
subs_to_m_and_eta_symmetrical = {
    Z2sym.E_symmetrical(h, l, 1, 1): _expr_E12,
    Z2sym.E_symmetrical(h, l, 1, 0): _expr_E1,
    Z2sym.cF2_symmetrical(h, l): _expr_cF2_1
}
subs_to_m_and_eta = subs_to_m_and_eta_symmetrical

In [16]:
expr_eta2 = Z2.diff(l, 2)/Z2
expr_eta2 = expr_eta2.expand()

expr = expr_eta2.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

\left<\eta\right>*(-3*\lambda + h**2)/(2*\lambda**2) + \left<m\right>*h*(\lambda + 1)/\lambda**2 + (2*\lambda**2 + 1)/(2*\lambda**2) - (2*\lambda*\left<m\right>*h + \lambda + h**2)*E_symmetrical(h, \lambda, 0, 0)/(2*\lambda**2*E_symmetrical(h, \lambda, 1, 1))

\frac{\left<\eta\right> \left(- 3 \lambda + h^{2}\right)}{2 \lambda^{2}} + \frac{\left<m\right> h \left(\lambda + 1\right)}{\lambda^{2}} + \frac{2 \lambda^{2} + 1}{2 \lambda^{2}} - \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{2 \lambda^{2} {\cal E}_{12}\left(h, \lambda\right)}


In [17]:
expr_m_eta = Z2.diff(h, l)/(2*Z2)
expr_m_eta = expr_m_eta.expand()

expr = expr_m_eta.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

-\left<\eta\right>*(\lambda + h**2)/(2*\lambda*h) - \left<m\right>/\lambda + (2*\lambda*\left<m\right>*h + \lambda + h**2)*E_symmetrical(h, \lambda, 0, 0)/(2*\lambda*h*E_symmetrical(h, \lambda, 1, 1)) - 1/(2*\lambda*h)

- \frac{\left<\eta\right> \left(\lambda + h^{2}\right)}{2 \lambda h} - \frac{\left<m\right>}{\lambda} + \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{2 \lambda h {\cal E}_{12}\left(h, \lambda\right)} - \frac{1}{2 \lambda h}


In [18]:
_E = Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1)
subs_to_m_eta_symmetrical = {
    _E: sp.solve(expr_m_eta.subs(subs_to_m_and_eta)-Z2sym.m_eta, _E)[0]
}
subs_to_m_eta = subs_to_m_eta_symmetrical

In [19]:
subs_to_upsilon = {Z2sym.m_eta: m*(1-2*Z2sym.upsilon)}

In [20]:
expr = expr_eta2.subs(subs_to_m_and_eta).subs(subs_to_m_eta).expand()
subs_to_eta2 = {l: sp.solve(expr-Z2sym.eta2, l)[0]}
expr = sp.Eq(l/2*(Z2sym.eta2-1), sp.expand(l/2*(expr.subs(subs_to_upsilon)-1)))

display(expr)
sp.print_latex(expr)

Eq(\lambda*({\left<\eta^2\right>} - 1)/2, \Upsilon*\left<m\right>*h - \left<\eta\right>)

\frac{\lambda \left({\left<\eta^2\right>} - 1\right)}{2} = \Upsilon \left<m\right> h - \left<\eta\right>


In [21]:
for moment, expr_moment in zip([m, eta], [expr_m, expr_eta]):
    for arg in [h, l]:
        expr = expr_moment.diff(arg).expand().subs(subs_to_m_and_eta).expand()
        expr = expr.subs(subs_to_m_eta).factor().expand()
        expr = expr.subs(subs_to_eta2).factor().expand()
        display(sp.Eq(sp.Derivative(moment, arg), expr))

Eq(Derivative(\left<m\right>, h), \left<\eta\right> - 2*\left<m\right>**2 - 2*\left<m\right>/h + 1)

Eq(Derivative(\left<m\right>, \lambda), -\left<\eta\right>*\left<m\right> + {\left<m\eta\right>})

Eq(Derivative(\left<\eta\right>, h), -2*\left<\eta\right>*\left<m\right> + 2*{\left<m\eta\right>})

Eq(Derivative(\left<\eta\right>, \lambda), -\left<\eta\right>**2 + {\left<\eta^2\right>})

### Численная проверка

In [22]:
def calc_Z2_symmetrical_integrate(args):
    h, l = args
    return Z2num.calc_Z2_integrate(0, h, h, l)

In [23]:
f_expr_m = sp.lambdify((h, l), expr_m, Z2sym.num_module_norm)
f_expr_eta = sp.lambdify((h, l), expr_eta, Z2sym.num_module_norm)

args = 4, 5
moments = nd.Gradient(calc_Z2_symmetrical_integrate)(args)/calc_Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0]/2)
print(f_expr_m(*args))
print(z2.m)
print()
print(moments[1])
print(f_expr_eta(*args))
print(z2.eta)

0.8371722011551646
0.8371718561570379
0.8371718561570379

0.8546061112004999
0.8546056972028968
0.8546056972028968


In [24]:
f_expr_cF2_1 = sp.lambdify((h, l, m, eta), _expr_cF2_1, Z2sym.num_module)
f_expr_cF2_2 = sp.lambdify((h, l, m, eta), _expr_cF2_2, Z2sym.num_module)

args = 1, 1.6
args_moments = f_expr_m(*args), f_expr_eta(*args)

print(Z2num.cF2_symmetrical(*args))
print(f_expr_cF2_1(*args, *args_moments))
print(f_expr_cF2_2(*args, *args_moments))

8.34716277372662
8.347162773726618
8.34716277372662


In [25]:
f_expr_m_eta = sp.lambdify((h, l), expr_m_eta, Z2sym.num_module_norm)
f_expr_eta2 = sp.lambdify((h, l), expr_eta2, Z2sym.num_module_norm)

args = 3.4, 1.9
moments = nd.Hessian(calc_Z2_symmetrical_integrate)(args)/calc_Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0, 1]/2)
print(moments[1, 0]/2)
print(f_expr_m_eta(*args))
print(z2.m_eta)
print()
print(moments[1, 1])
print(f_expr_eta2(*args))
print(z2.eta2)

0.5815164055484547
0.5815164055484547
0.5815162841798034
0.5815162841798025

0.5911376785139352
0.5911371476339244
0.5911371476339227


## Предельные случаи 

### $\lambda \rightarrow 0$

In [26]:
expr = Z2.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 1).factor()
expr = sp.limit(expr, l, 0).expand()
display(expr)

expr.rewrite(sp.sin).factor()

4*pi**2*exp(2*h)/h**2 - 8*pi**2/h**2 + 4*pi**2*exp(-2*h)/h**2

8*pi**2*(cosh(2*h) - 1)/h**2

In [27]:
expr = expr_m.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 2).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

subs_to_m_llb = {
    sp.exp(2*h): sp.solve(expr.expand()-m, sp.exp(2*h))[0]
}

expr.expand().rewrite(sp.tanh).factor().expand()

(h*exp(2*h) + h - exp(2*h) + 1)/(h*(exp(h) - 1)*(exp(h) + 1))

1/tanh(h) - 1/h

In [28]:
expr = expr_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 3).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr.expand().subs(subs_to_m_llb).factor()

(h*exp(2*h) + h - exp(2*h) + 1)**2/(h**2*(exp(h) - 1)**2*(exp(h) + 1)**2)

\left<m\right>**2

In [29]:
expr = expr_m_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

(h*exp(2*h) + h - exp(2*h) + 1)*(h**2*exp(2*h) - h**2 - 2*h*exp(2*h) - 2*h + 2*exp(2*h) - 2)/(h**3*(exp(h) - 1)**2*(exp(h) + 1)**2)

-2*\left<m\right>**2/h + \left<m\right>

- \frac{2 \left<m\right>^{2}}{h} + \left<m\right>


In [30]:
expr = expr_eta2.rewrite(sp.exp).factor()
expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

(h**4*exp(4*h) - 2*h**4*exp(2*h) + h**4 - 4*h**3*exp(4*h) + 4*h**3 + 10*h**2*exp(4*h) + 4*h**2*exp(2*h) + 10*h**2 - 12*h*exp(4*h) + 12*h + 6*exp(4*h) - 12*exp(2*h) + 6)/(h**4*(exp(h) - 1)**2*(exp(h) + 1)**2)

6*\left<m\right>**2/h**2 - 4*\left<m\right>/h + 1

\frac{6 \left<m\right>^{2}}{h^{2}} - \frac{4 \left<m\right>}{h} + 1


In [31]:
# expr = expr_m.diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr.expand().subs(subs_to_m_llb).factor().expand()

In [32]:
# expr = (expr_eta).diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# expr

In [33]:
# expr = (expr_m_eta/2).diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 6).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)

# expr = expr/m - (m-2*m**2/h)/m**2*(m - m * m * m - 2 * m * m/h)
# expr = expr.expand()
# display(expr)
# display(expr.replace(m, h/3))

In [34]:
# expr = expr_eta2.diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 7).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)
# display(expr.replace(m, h/3))

### $h \rightarrow 0$

In [35]:
expr = Z2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h).replace(h, 0).expand()
display(expr)

expr = expr.rewrite(sp.sin).factor()
display(expr)
sp.print_latex(expr)

8*pi**2*exp(\lambda)/\lambda - 8*pi**2*exp(-\lambda)/\lambda

16*pi**2*sinh(\lambda)/\lambda

\frac{16 \pi^{2} \sinh{\left(\lambda \right)}}{\lambda}


In [36]:
expr = expr_m.rewrite(sp.exp).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.replace(h, 0)
display(expr)

0

In [37]:
expr = expr_eta.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h)
expr = expr.replace(h, 0).factor()
display(expr)

subs_to_eta_prmg = {
    sp.exp(2*l): sp.solve(expr.expand()-eta, sp.exp(2*l))[0]
}

expr = expr.expand().rewrite(sp.tanh).factor().expand()
display(expr)
sp.print_latex(expr)

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(exp(\lambda) - 1)*(exp(\lambda) + 1))

1/tanh(\lambda) - 1/\lambda

\frac{1}{\tanh{\left(\lambda \right)}} - \frac{1}{\lambda}


In [38]:
expr = ((1-expr_m_eta/expr_m)/2)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(2*\lambda*exp(2*\lambda) - exp(2*\lambda) + 1))

\left<\eta\right>/(\lambda*(\left<\eta\right> + 1))

In [39]:
expr = expr_eta2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor().expand()

(\lambda**2*exp(2*\lambda) - \lambda**2 - 2*\lambda*exp(2*\lambda) - 2*\lambda + 2*exp(2*\lambda) - 2)/(\lambda**2*(exp(\lambda) - 1)*(exp(\lambda) + 1))

1 - 2*\left<\eta\right>/\lambda

In [40]:
expr = expr_m/h
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

(2*\lambda*exp(2*\lambda) - exp(2*\lambda) + 1)/(3*\lambda*(exp(\lambda) - 1)*(exp(\lambda) + 1))

(\left<\eta\right> + 1)/3

In [41]:
# expr = (expr_m/h).diff(h).factor()
# expr = symbase.LHopital(expr, h, 5)
# expr = expr.rewrite(sp.exp).replace(h, 0)
# display(expr)

In [42]:
expr = expr_eta.diff(h).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0)
display(expr)

0

In [43]:
# expr = ((1-expr_m_eta/expr_m)/2).diff(h).factor()
# expr = symbase.LHopital(expr.factor(), h, 6)
# expr = expr.rewrite(sp.exp).replace(h, 0).factor()
# display(expr)

## Генерация кода C++ для реализации метода Ньютона

In [44]:
subs = {
    Z2sym.cF2_symmetrical(h, l): sp.Symbol("cF2_norm"),
    Z2sym.E_symmetrical(h, l, 1, 1): sp.Symbol("E12_norm"),
    Z2sym.E_symmetrical(h, l, 1, 0): sp.Symbol("E1_norm"),
    Z2sym.E_symmetrical(h, l, 0, 0): sp.Symbol("E_norm"),
    m: sp.Symbol("m"), eta: sp.Symbol("eta"),
    sp.exp(-2*h): sp.Symbol("_exp_2h"),
    sp.exp(-2*l): sp.Symbol("_exp_2l"),
    l: sp.Symbol("l")
}


def optimize(expr):
    expr = symbase.optimize_pow(expr, {l: "l"})
    expr = expr.subs(subs)
    return expr

In [45]:
print(symbase.get_ccode(optimize(.5*(1-expr_m_eta/m)).factor()))

0.5*(-0.125*M_SQRT2*E12_norm*(h*h*h)*l + 0.125*M_SQRT2*E12_norm*h*(l*l) + 0.25*M_SQRT2*E1_norm*(h*h)*(l*l) + 0.25*M_SQRT2*E1_norm*(l*l*l) - 0.25*M_SQRT2*E_norm*h*(l*l*l) + 0.25*cF2_norm*(h*h*h*h)*sqrt_l - 0.5*cF2_norm*(h*h)*(l*l)*sqrt_l - 0.5*cF2_norm*(h*h)*l*sqrt_l + 1.0*cF2_norm*h*(l*l*l)*m*sqrt_l - 0.5*cF2_norm*(l*l*l)*sqrt_l - 0.25*cF2_norm*(l*l)*sqrt_l)/(cF2_norm*h*(l*l*l)*m*sqrt_l)


## Решение обратной задачи

### Прямая задача

In [46]:
h = np.logspace(-5, 1, 100)
l = np.logspace(-5, 1, 100)

h, l = np.meshgrid(h, l)

In [47]:
m, eta = np.empty_like(h), np.empty_like(h)
upsilon, eta2 = np.empty_like(h), np.empty_like(h)
Z2_norm = np.empty_like(m)

for i in range(h.size):
    z2 = Z2num.calc_from_coeffs_symmetrical(h.flat[i], l.flat[i])
    m.flat[i], eta.flat[i] = z2.m, z2.eta
    upsilon.flat[i], eta2.flat[i] = z2.upsilon, z2.eta2
    Z2_norm.flat[i] = z2.Z2_norm

In [48]:
datlib.np2dat(
    "data/Z2.dat",
    "h l upsilon eta2 Z2_norm".split(),
    np.asarray([
        h, l, upsilon, eta2, Z2_norm
    ])
)

In [49]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm' data/Z2.dat -ln xy -to data/picts/pict.pdf 

### Обратная задача

In [50]:
m = np.linspace(0, 1, 100)
zeta = np.linspace(0, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [51]:
h, l = np.empty_like(m), np.empty_like(m)
upsilon, eta2, mhj = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mhj.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm

In [52]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta h l upsilon eta2 mhj psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, h, l, upsilon, eta2, mhj, psi0, Z2_norm
    ])
)

In [53]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm(m,zeta)' data/Z2.dat -to data/picts/pict.pdf 

### Асимптотика при $\left<m\right> \rightarrow 1$

In [54]:
m = np.linspace(.99, 1, 100)
zeta = np.linspace(.05, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [55]:
h, l = np.empty_like(m), np.empty_like(m)
upsilon, eta2, mhj = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical_by_scipy(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mhj.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm

/home/daiwery/projects/Z-analytic/python/Z2num.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  return .5/(1-m), .5/(1-eta)


In [56]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta h l upsilon eta2 mhj psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, h, l, upsilon, eta2, mhj, psi0, Z2_norm
    ])
)

In [57]:
# %%bash
# gplt3 -pm3d -U 'upsilon(m,eta)' data/Z2.dat -to data/picts/pict.pdf 

In [58]:
variables = 1-m, 1-eta
y_data = upsilon
indexes_nans = np.logical_not(np.isnan(h))

def func(_, *p_args, nans=True):    
    result = p_args[0]
    result += p_args[1]*variables[0] + p_args[2]*variables[1]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result
    

p0 = [0] + 2*[1]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

array([1.80037290e-06, 2.39256366e-04, 4.98311465e-01])

In [59]:
# x = 1 - m
# y = 1 - eta

# upsilon = y/2
# eta2 = 1 - 2*y
# m_par_2 = 1 - 2*x
# psi0 = -y/2

# Ферромагнетик в несимметричном случае

## Выражение для $Z^{(2)}$

### Аргументы

In [60]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t = sp.Symbol("t")

In [61]:
arg_exp = t**2/(2*l)*hi/hj + s1*t
bound = hj+s2*l

arg_exp = arg_exp.replace(t, t-s1*l*hj/hi).expand()
bound += s1*l*hj/hi

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*l*hj/hi))
bound /= sp.sqrt(2*l*hj/hi)

display(arg_exp)
display(bound)

-\lambda*h_2/(2*h_1) + t**2

sqrt(2)*(\sigma_1*\lambda*h_2/h_1 + \sigma_2*\lambda + h_2)/(2*sqrt(\lambda*h_2/h_1))

In [62]:
expr = bound**2 + (arg_exp - t**2)
expr = expr.expand()
expr -= (hj**2+l**2)/(2*l*hj/hi)
expr = expr.expand()

display(expr)

\sigma_1*\sigma_2*\lambda + \sigma_1*h_2 + \sigma_2*h_1

### Производные $\cal{F}^{(2)}$

In [63]:
display(Z2sym.E_not_symmetrical(hi, hj, l, 1, 1).diff(l).rewrite(sp.exp))
display(Z2sym.E_not_symmetrical(hi, hj, l, 1, 1).rewrite(sp.exp).diff(l))

exp(\lambda)*exp(h_1)*exp(h_2) + exp(\lambda)*exp(-h_1)*exp(-h_2) + exp(-\lambda)*exp(h_1)*exp(-h_2) + exp(-\lambda)*exp(-h_1)*exp(h_2)

exp(\lambda)*exp(h_1)*exp(h_2) + exp(\lambda)*exp(-h_1)*exp(-h_2) + exp(-\lambda)*exp(h_1)*exp(-h_2) + exp(-\lambda)*exp(-h_1)*exp(h_2)

In [64]:
# display(Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hi).rewrite(sp.exp).expand())
# display(Z2sym.cF2_not_symmetrical(hi, hj, l).rewrite(sp.exp).diff(hi).expand())

### Численная проверка

In [65]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
cF2_diff_hi = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hi), Z2sym.num_module)
cF2_diff_hj = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(hj), Z2sym.num_module)
cF2_diff_l = sp.lambdify((hi, hj, l), Z2sym.cF2_not_symmetrical(hi, hj, l).diff(l), Z2sym.num_module)

args = 1, -2, -1.4
gradient = nd.Gradient(lambda args: Z2num.cF2_not_symmetrical(*args))(args)

print(gradient[0])
print(cF2_diff_hi(*args))
print()
print(gradient[1])
print(cF2_diff_hj(*args))
print()
print(gradient[2])
print(cF2_diff_l(*args))

16.264777260931098
16.264776699753636

-14.133145019126196
-14.13314313537626

-14.618964379619326
-14.618962644973239


In [66]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
sigmas_args = 1, 1
E_diff_hi = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(hi), Z2sym.num_module)
E_diff_hj = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(hj), Z2sym.num_module)
E_diff_l = sp.lambdify((hi, hj, l), Z2sym.E_not_symmetrical(hi, hj, l, *sigmas_args).diff(l), Z2sym.num_module)

args = 1, 1.8, 1.4
sigmas_args = list(map(bool, sigmas_args))

gradient = nd.Gradient(lambda args: Z2num.E_not_symmetrical(*args, *sigmas_args))(args)

print(gradient[0])
print(E_diff_hi(*args))
print()
print(gradient[1])
print(E_diff_hj(*args))
print()
print(gradient[2])
print(E_diff_l(*args))

66.8777425547184
66.87774255471518

66.00172559925015
66.0017255992518

67.5925427993252
67.59254279932306


## Моменты

In [67]:
hi, hj, l = Z2sym.hi, Z2sym.hj, Z2sym.l
Z2 = sp.sqrt(2)*(2*sp.pi)**2/sp.sqrt(hi*hj*l)*Z2sym.cF2_not_symmetrical(hi, hj, l)
Z2

4*sqrt(2)*pi**2*cF2_not_symmetrical(h_1, h_2, \lambda)/sqrt(\lambda*h_1*h_2)

### Симметричный случай

In [68]:
h, l = Z2sym.h, Z2sym.l

In [69]:
expr_hessian_symmetrical = sp.zeros(3, 3)
for i in range(3):
    for j in range(3):
        arg1, arg2 = [hi, hj, l][i], [hi, hj, l][j]
        expr_hessian_symmetrical[i, j] = Z2.diff(arg1, arg2)/Z2
        expr_hessian_symmetrical[i, j] = expr_hessian_symmetrical[i, j].expand().subs({hi: h, hj: h})

expr = expr_hessian_symmetrical
expr = expr.subs(subs_to_m_and_eta_symmetrical).expand()
expr = expr.subs(subs_to_m_eta_symmetrical)
expr = sp.Matrix(3, 3, lambda i, j: expr[i, j].factor()).expand()

expr[2, 2] = expr[2, 2].subs(subs_to_eta2).factor()

for i in range(3):
    for j in range(3):
        if expr[i, j] != Z2sym.m_eta:
            expr[i, j] = expr[i, j].subs(subs_to_upsilon).expand()

expr

Matrix([
[2*\Upsilon*\lambda*\left<m\right>/h - 2*\left<m\right>/h + 1,     -2*\Upsilon*\lambda*\left<m\right>/h + \left<\eta\right>,  {\left<m\eta\right>}],
[    -2*\Upsilon*\lambda*\left<m\right>/h + \left<\eta\right>, 2*\Upsilon*\lambda*\left<m\right>/h - 2*\left<m\right>/h + 1,  {\left<m\eta\right>}],
[                                        {\left<m\eta\right>},                                         {\left<m\eta\right>}, {\left<\eta^2\right>}]])

#### Момент $\Psi$

In [70]:
from sympy.vector import CoordSys3D, Del

N = CoordSys3D("N")
delop = Del()

vhi, vhj, vh3 = sp.symbols("h_1 h_2 h_3")
vh = vhi*N.i + vhj*N.j + vh3*N.k

nk1, nk2, nk3 = sp.symbols("n_{1K} n_{2K} n_{3K}")
nk = nk1*N.i + nk2*N.j + nk3*N.k

In [71]:
gamma1, gamma2 = sp.symbols("\\gamma_1 \\gamma_2", positive=True)

In [72]:
v1 = vh + gamma1*nk
v2 = vh + gamma2*nk

v1_par_2 = v1.dot(v2)/v2.magnitude()
v1_per_2 = sp.sqrt(v1.dot(v1) - v1_par_2**2)
v2_magnitude = v2.magnitude()

In [73]:
def _diff_func(func, args): 
    if len(args) == 0:
        return func

    arg = args[0]
    if isinstance(func, sp.Function):
        func = func.fdiff(arg)
        return _diff_func(func, args[1:])
    if isinstance(func, sp.Subs):
        return sp.Subs(_diff_func(func.args[0], args), *func.args[1:])
    if isinstance(func, sp.Derivative):
        return sp.Derivative(_diff_func(func.args[0], args), *func.args[1:])

In [74]:
Z = sp.Function("Z")(v1_per_2, v1_par_2, v2_magnitude, l)

def two_diff_func(func, arg1, arg2):
    args = func.args
    result = 0
    for i, arg in enumerate(args):
        result += arg.diff(arg1, arg2).subs({gamma2: 0}).factor()*_diff_func(func, [i+1])

        factor = arg.diff(arg1).subs({gamma2: 0})
        for j, j_arg in enumerate(args):
            term = (factor*j_arg.diff(arg2)).subs({gamma2: 0}).factor()
            result += term*_diff_func(func, [i+1, j+1])
    return result

expr_psi = (two_diff_func(Z, gamma1, gamma1).diff(l) - two_diff_func(Z, gamma1, gamma2)).subs({gamma1: 0, gamma2: 0})
expr_psi = expr_psi.subs(vh.dot(vh), h**2).expand()

def v_per_diff_eval(*args):
    func = args[0]
    diff_args = args[1:]
    per_arg = func.args[0]
    non_per_diff_args = tuple(filter(lambda item: item[0] != per_arg, diff_args))
    per_diff_args = tuple(filter(lambda item: item[0] == per_arg, diff_args))

    if len(per_diff_args) == 0:
        return sp.Derivative(*args)
    
    _, n = per_diff_args[0]

    if n == 2:
        func = func - _diff_func(func, (2, 2))
        func /= 2
    elif n == 1:
        return 0
    else:
        return sp.Derivative(*args)

    if len(non_per_diff_args) > 0:
        func = sp.Derivative(func, *non_per_diff_args)

    return func

expr_psi = expr_psi.replace(sp.Derivative, v_per_diff_eval).doit()

expr_psi = expr_psi.replace(sp.Function("Z"), lambda *args: Z2.subs(dict(zip([hi, hj, l], args[1:])))).doit()
expr_psi = (expr_psi/Z2.subs({hi: h, hj: h})).expand()
expr_psi = expr_psi.factor()
expr_psi = expr_psi.replace(sp.sqrt(h**2), h)
expr_psi

-(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)*(24*\lambda**(15/2)*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 16*\lambda**(13/2)*h**2*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(13/2)*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 8*\lambda**(11/2)*h**4*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(11/2)*h**2*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(9/2)*h**4*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) - 2*\lambda**(7/2)*h**6*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*sqrt(2)*\lambda**7*h**4*E_symmetrical(h, \lambda, 0, 0) - 6*sqrt(2)*\lambda**7*h**4*E_symmetrical(h, \lambda, 1, 1) - 6*sqrt(2)*\lambda**7*(h**2)**(3/2)*E_symmetrical(h, \lambda, 1, 0) + 2*sqrt(2)*\lambda**6*h**6*E_symmetrical(h, \lambda, 0, 0) - 2*sqrt(2)*\lambda**6*h**6*E_symmetr

In [75]:
expr_psi_nk = expr_psi.args[-2]
display(expr_psi_nk)

expr = expr_psi_nk-(3*(vh.dot(nk)**2)-vh.dot(vh))
expr = expr.expand()
expr = expr.subs({nk1**2: 1-nk2**2-nk3**2})
expr.expand()

2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2

0

In [76]:
expr_psi0 = expr_psi/expr_psi_nk
expr_psi0 = expr_psi0.expand()
expr_psi0 = expr_psi0.replace(h, sp.Abs(h)).replace(sp.Abs(h), h)
expr_psi0 = expr_psi0.subs(subs_to_m_and_eta_symmetrical).expand()
expr_psi0 = expr_psi0.subs(subs_to_m_eta_symmetrical).factor().expand()
expr_psi0 = expr_psi0.subs(subs_to_eta2).factor().expand()
expr_psi0 = expr_psi0.subs(subs_to_upsilon).expand()
expr_psi0

3*\Upsilon*\left<m\right>/(2*h**3) + {\left<\eta^2\right>}/(4*h**2) - 1/(4*h**2)

### Численная проверка

In [77]:
def calc_Z2_integrate(hi, hj, l):
    hjz = np.linalg.norm(hj)
    hiz = hi.dot(hj)/hjz
    hiy = np.sqrt(np.abs(hi.dot(hi) - hiz**2))
    return Z2num.calc_Z2_integrate(hiy, hiz, hjz, l)

In [78]:
hi = np.asarray([0, 3, 2])
hj = np.asarray([0, 0, 1])
l = 2

print(calc_Z2_integrate(hi, hj, l))
print(Z2num.calc_Z2_integrate(hi[1], hi[2], hj[2], l))

2062.17766247412
2062.17766247412


In [79]:
def Z2_not_symmetrical_integrate(args):
    hi, hj, l = args
    return Z2num.calc_Z2_integrate(0, hi, hj, l)

In [80]:
def calc_Z2_symmetrical_with_nk_integrate(h, l, gamma1, gamma2, nk):
    h = np.asarray([0, 0, h])
    hi = h + gamma1*nk
    hj = h + gamma2*nk
    return calc_Z2_integrate(hi, hj, l)

#### Симметричный случай

In [81]:
h, l = Z2sym.h, Z2sym.l
f_expr_mh2 = sp.lambdify((h, l), expr_hessian_symmetrical[0, 0], Z2sym.num_module_norm)

args = 1, 1.2
args_not_symmetrical = args[0], args[0], args[1]

moments = nd.Hessdiag(Z2_not_symmetrical_integrate)(args_not_symmetrical)/Z2_not_symmetrical_integrate(args_not_symmetrical)

print(moments[0])
print(moments[1])
print(f_expr_mh2(*args))

0.4002859831723324
0.400285983172324
0.40028594111272375


In [82]:
expr_psi

-(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)*(24*\lambda**(15/2)*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 16*\lambda**(13/2)*h**2*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(13/2)*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 8*\lambda**(11/2)*h**4*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(11/2)*h**2*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*\lambda**(9/2)*h**4*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) - 2*\lambda**(7/2)*h**6*(h**2)**(3/2)*cF2_symmetrical(h, \lambda) + 6*sqrt(2)*\lambda**7*h**4*E_symmetrical(h, \lambda, 0, 0) - 6*sqrt(2)*\lambda**7*h**4*E_symmetrical(h, \lambda, 1, 1) - 6*sqrt(2)*\lambda**7*(h**2)**(3/2)*E_symmetrical(h, \lambda, 1, 0) + 2*sqrt(2)*\lambda**6*h**6*E_symmetrical(h, \lambda, 0, 0) - 2*sqrt(2)*\lambda**6*h**6*E_symmetr

In [83]:
h, l = Z2sym.h, Z2sym.l
f_expr_psi = sp.lambdify((vhi, vhj, vh3, nk1, nk2, nk3, h, l), expr_psi, Z2sym.num_module_norm)

h, l = 1.4, 1.6
vh = np.asarray([0, 0, h])
nk = np.asarray([0, 1, 1.])
nk /= np.linalg.norm(nk)

print(f_expr_psi(*vh, *nk, h, l))

z2 = calc_Z2_symmetrical_integrate((h, l))

hessian = nd.Hessian(lambda args: calc_Z2_symmetrical_with_nk_integrate(h, l, *args, nk))((0, 0))
hessdiag_l = nd.Derivative(lambda l: nd.Hessdiag(lambda args: calc_Z2_symmetrical_with_nk_integrate(h, l, *args, nk))((0, 0)))(l)

psi = hessdiag_l[0]/z2-hessian[0, 1]/z2
print(psi)

z2 = Z2num.calc_from_coeffs_symmetrical(h, l)
print((1-z2.eta)*z2.m**3*(0.46134-1.3836*(vh/np.linalg.norm(vh)).dot(nk)**2))

-0.015561919331182525
-0.015562017789831378
-0.015359404914857131
